In [ ]:
import polars as pl
import seaborn.objects as so

In [ ]:
nps = pl.read_csv('data/nps.csv', try_parse_dates=True)
nps_stages = pl.read_csv('data/nps_stages.csv')
noaa = pl.read_csv('data/noaa.csv', try_parse_dates=True)

In [ ]:
def day_of_year(year: pl.Series, date: pl.Series):
    jan1 = pl.format('{}-01-01', year).str.strptime(pl.Date, '%Y-%m-%d')
    doy = (date - jan1) / pl.duration(days=1)
    return doy

df = (
    pl.concat([
        noaa.select(['year', 'date']).with_columns(place=pl.lit('kyoto')),
        nps.filter(pl.col('stage') == 'Peak Bloom').select(['year', 'date']).with_columns(place=pl.lit('dc'))
    ])
    .with_columns(doy = day_of_year(pl.col('year'), pl.col('date')))
)

df

In [ ]:
(
    so.Plot(df.to_pandas(), x='doy', color='place')
    .add(so.Bars(), so.Hist())
)

In [ ]:
(
    so.Plot(df.to_pandas(), x='year', y='doy', color='place')
    .add(so.Dots())
)

In [ ]:

kyoto = (
    noaa
    .select(['year', 'day_of_year', 'temp_reconstructed', 'temp_observed'])
    .melt(id_vars=['year', 'day_of_year'])
    .with_columns(pl.col('value').cast(float))
)

(
    so.Plot(kyoto.to_pandas(), x='day_of_year', y='value', color='variable')
    .add(so.Dots())
)
